# **DIV2K Single Image Super-Resolution Challenge**

$\quad$

[1] C. Ledig *et al.*, Photo-realistic single image super-resolution using a generative adversarial network, CVPR, 2017.


## **Install MMEditing**

In [1]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# Install dependencies
!pip install cython mmcv
# # Install mmcv from source
# !rm -rf mmcv
# !git clone https://github.com/open-mmlab/mmcv.git
# %cd mmcv
# !MMCV_WITH_OPS=1 pip install .
# %cd ..

# Install MMEditing
!rm -rf mmediting
!git clone https://github.com/open-mmlab/mmediting.git
%cd mmediting
!pip install -r requirements.txt
!pip install -v -e .
# /home/MSAI/jong119/.conda/envs/ai6126p2/bin/pip install -r requirements.txt
# %cd ../..

Cloning into 'mmediting'...
remote: Enumerating objects: 4796, done.
remote: Total 4796 (delta 0), reused 0 (delta 0), pack-reused 4796
Receiving objects: 100% (4796/4796), 3.61 MiB | 28.85 MiB/s, done.
Resolving deltas: 100% (3310/3310), done.
/content/mmediting
Created temporary directory: /tmp/pip-ephem-wheel-cache-fmimf87p
Created temporary directory: /tmp/pip-req-tracker-erhzlim7
Created requirements tracker '/tmp/pip-req-tracker-erhzlim7'
Created temporary directory: /tmp/pip-install-mkt9l7zg
Obtaining file:///content/mmediting
  Added file:///content/mmediting to build tracker '/tmp/pip-req-tracker-erhzlim7'
    Running setup.py (path:/content/mmediting/setup.py) egg_info for package from file:///content/mmediting
    Running command python setup.py egg_info
    running egg_info
    creating mmedit.egg-info
    writing mmedit.egg-info/PKG-INFO
    writing dependency_links to mmedit.egg-info/dependency_links.txt
    writing requirements to mmedit.egg-info/requires.txt
    writing

In [3]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMEditing installation
import mmedit
print(mmedit.__version__)

1.7.0+cu101 True
0.5.0


### **Download images**


In [4]:
!mkdir -p data/DIV2K
%cd data/DIV2K

!wget https://www.dropbox.com/sh/e7opsbgu5ww1qe3/AAAvrcVCykCR2-G--e2H1WCxa?dl=1 --content-disposition

# unzip the downloaded file
!unzip -qq Mini-DIV2K.zip
!rm -f Mini-DIV2K.zip

# data should be in mmediting/data
%cd ../..
!ls

/content/mmediting/data/DIV2K
--2020-11-05 13:08:21--  https://www.dropbox.com/sh/e7opsbgu5ww1qe3/AAAvrcVCykCR2-G--e2H1WCxa?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:6019:1::a27d:401
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/dl/e7opsbgu5ww1qe3/AAAvrcVCykCR2-G--e2H1WCxa [following]
--2020-11-05 13:08:21--  https://www.dropbox.com/sh/dl/e7opsbgu5ww1qe3/AAAvrcVCykCR2-G--e2H1WCxa
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbcf00d9dfb922d64c380952d9d.dl.dropboxusercontent.com/zip_download_get/AmC1x3zcfDf7okDN27xXIJ1VCt9Mu6WCe29OqRUQfJAAU3wzorKWgUFCZpc2EqGgXYMmCWgDXGJE16rMTkHYr1dOxt4o77UVsvFuxq3EmsDoNQ?dl=1 [following]
--2020-11-05 13:08:22--  https://ucbcf00d9dfb922d64c380952d9d.dl.dropboxusercontent.com/zip_download_get/AmC1x3zcfDf7okDN27xXIJ1VCt9Mu6WCe29OqRUQfJAAU3wzorK

### **Inference**

In [5]:
# import cv2
# import matplotlib.pyplot as plt
# import mmcv
# import torch
# import torchvision

# from mmedit.models import build_model
# from mmcv.runner import load_checkpoint

# # Build model
# cfg = mmcv.Config.fromfile('configs/restorers/srcnn/srcnn_x4k915_g1_1000k_div2k.py')
# model = build_model(cfg.model).cuda()
# load_checkpoint(model, './checkpoint/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth', map_location='cuda')

# # Read the images
# img_LR = mmcv.imread('./data/Set5/LR/butterfly.png', channel_order='rgb')
# img_HR = mmcv.imread('./data/Set5/GT/butterfly.png', channel_order='rgb')

# # Cast the input image as a PyTorch FloatTensor
# img_SR = torch.from_numpy(img_LR).cuda().float().permute(2, 0, 1).unsqueeze(0)
# img_SR = img_SR / 255  # Input range = [0, 1]

# # Compute the output and cast it back to a numpy array
# with torch.no_grad():
#   img_SR = model.forward_test(img_SR)['output']
# img_SR = torch.clamp(img_SR, 0, 1)  # Output range = [0, 1]
# img_SR = img_SR.squeeze(0).permute(1, 2, 0).numpy()

# # Let's compare the result with the LR and HR images
# fig = plt.figure(figsize=(15, 12))
# ax1 = fig.add_subplot(1, 3, 1)  # 1 row, 3 columns, the top left image  
# plt.title('LR image', fontsize=16)
# ax1.axis('off')
# ax2 = fig.add_subplot(1, 3, 2)  # 2 row, 3 columns, the top right image 
# plt.title('SRCNN output', fontsize=16)
# ax2.axis('off')
# ax3 = fig.add_subplot(1, 3, 3)  # 2 row, 3 columns, the bottom right image  
# plt.title('HR image', fontsize=16)
# ax3.axis('off')
# ax1.imshow(img_LR)
# ax2.imshow(img_SR)
# ax3.imshow(img_HR)

The output is much better than the LR input! But it is still a bit blurry, why is that? We will tell you the reason later.

### **Filter Visualization**

SRCNN is a convolutional neural network. That means, each layer consists of multiple *convolution filters*. You may wonder what the learned filters are. Is there any relation between the learned filters and the hand-designed filters, such as Sobel filter, Gaussian filter, etc.? Here, we visualize the filters of the first layer to provide an intuition. This layer directly operates on the input image.

In [6]:
# import mmcv
# import torchvision
# from mmedit.models import build_model
# from mmcv.runner import load_checkpoint

# cfg = mmcv.Config.fromfile('configs/restorers/srcnn/srcnn_x4k915_g1_1000k_div2k.py')
# model = build_model(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
# load_checkpoint(model, './checkpoint/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth', map_location='cpu')
# conv1_weight = model.generator.conv1.weight.detach()
# n_out, n_in, h, w = conv1_weight.size()
# conv1_weight = conv1_weight.view(-1, 1, h, w)
# grid_img = torchvision.utils.make_grid(conv1_weight, normalize=True, scale_each=True, nrow=64)
# fig = plt.figure(figsize=(40, 30))
# plt.axis('off')
# plt.imshow(grid_img.permute(1, 2, 0))


We can see that the learned fitlers are diverse. For example, we can see both isotropic (uniform in all directions) and anisotropic (directional) filters. The outputs of these filters are then fed to the next layer.

While it is easy to interpret the meaning (e.g. edge detection, blurring) of the filters when they are applied to the image, when going deeper, the filters become non-interpretable.

### **Blurring kernel**

As we discussed, the above output is still blurry. Why is that?

$\quad$

The performance of SRCNN (and many other data-driven methods) strongly depends on the training data. When the training data and test data have different distributions, the performance of such methods deteriorate significantly. In fact, the SRCNN we used above is trained with no blurring ($K=I$), but the input test image is blurred by a Gaussian kernel before downsampling. 

$\quad$

In order to improve the output quality, we should train a model using the same degradations. In the following, we will now finetune our pre-trained SRCNN using the sampling degradations. You will learn how to use MMEditing to train a restorer on a customized dataset.

## **Finetune SRCNN on A Customized Dataset**

To train a new restorer using MMEditing, there are usually three things to do:

1.   Support a new dataset
2.   Modify the config
3.   Train a new restorer

### **Support a new dataset**

To support a new dataset for paired training, there are two things you need to prepare.

1. Download the LR images and the corresponding HR images. They share the same file name but are put in different directories.

2. Create an annotation file for your dataset. The annoataion file contains all the image names and gt image shape. For example:

```
0001_s001.png (480,480,3)
0001_s002.png (480,480,3)
...
```

As an example, we will use a tiny dataset obtained from DIV2K. Let's create the annotation file for this training data.


In [7]:
import glob
import cv2

gt_paths = sorted(glob.glob('./data/DIV2K/Train/HR/*.png'))
with open('data/DIV2K/Train/training_ann.txt', 'w') as f:
  for gt_path in gt_paths:
    filename = gt_path.split('/')[-1]
    img = cv2.imread(gt_path,1)
    h, w, c = img.shape
    line = f'{filename} ({h},{w},{c})\n'
    f.write(line)

# Let's make sure the annotations are correct
with open('data/DIV2K/Train/training_ann.txt', 'r') as f:
  file_content = f.read()
  print(file_content)

0001.png (1848,2040,3)
0002.png (1356,2040,3)
0003.png (1344,2040,3)
0004.png (1356,2040,3)
0005.png (1356,2040,3)
0006.png (1524,2040,3)
0007.png (1644,2040,3)
0008.png (1356,2040,3)
0009.png (2040,1356,3)
0010.png (1356,2040,3)
0011.png (1368,2040,3)
0012.png (1536,2040,3)
0013.png (1392,2040,3)
0014.png (768,2040,3)
0015.png (960,2040,3)
0016.png (1356,2040,3)
0017.png (1992,2040,3)
0018.png (1356,2040,3)
0019.png (1272,2040,3)
0020.png (1356,2040,3)
0021.png (1356,2040,3)
0022.png (1356,2040,3)
0023.png (1356,2040,3)
0024.png (1356,2040,3)
0025.png (1224,2040,3)
0026.png (1356,2040,3)
0027.png (1356,2040,3)
0028.png (1356,2040,3)
0029.png (2040,1644,3)
0030.png (1224,2040,3)
0031.png (2040,1284,3)
0032.png (1428,2040,3)
0033.png (1356,2040,3)
0034.png (1320,2040,3)
0035.png (2040,1248,3)
0036.png (1356,2040,3)
0037.png (2040,1356,3)
0038.png (1356,2040,3)
0039.png (1140,2040,3)
0040.png (1356,2040,3)
0041.png (1356,2040,3)
0042.png (1356,2040,3)
0043.png (1332,2040,3)
0044.png (121

In [8]:
# import glob
# gt_paths = sorted(glob.glob('./data/DIV2K/GT/*.png'))
# with open('data/training_ann.txt', 'w') as f:
#   for gt_path in gt_paths:
#     filename = gt_path.split('/')[-1]
#     line = f'{filename} (480,480,3)\n'
#     f.write(line)

# # Let's make sure the annotations are correct
# with open('data/training_ann.txt', 'r') as f:
#   file_content = f.read()
#   print(file_content)

### **Modify the configuration file**

The next step is to modify the configuration file for training. In this tutorial, we finetune a pre-trained restorer to accelerate the process.

In [9]:
from mmcv import Config
# Load the original config
# cfg = Config.fromfile('configs/restorers/srcnn/srcnn_x4k915_g1_1000k_div2k.py')
cfg = Config.fromfile('configs/restorers/srresnet_srgan/msrresnet_x4c64b16_g1_1000k_div2k.py')
# cfg = Config.fromfile('configs/restorers/srresnet_srgan/srgan_x4c64b16_g1_1000k_div2k.py') # too many params

print(f'Config:\n{cfg.pretty_text}')  # Show the config

Config:
exp_name = 'msrresnet_x4c64b16_g1_1000k_div2k'
scale = 4
model = dict(
    type='BasicRestorer',
    generator=dict(
        type='MSRResNet',
        in_channels=3,
        out_channels=3,
        mid_channels=64,
        num_blocks=16,
        upscale_factor=4),
    pixel_loss=dict(type='L1Loss', loss_weight=1.0, reduction='mean'))
train_cfg = None
test_cfg = dict(metrics=['PSNR', 'SSIM'], crop_border=4)
train_dataset_type = 'SRAnnotationDataset'
val_dataset_type = 'SRFolderDataset'
train_pipeline = [
    dict(
        type='LoadImageFromFile',
        io_backend='disk',
        key='lq',
        flag='unchanged'),
    dict(
        type='LoadImageFromFile',
        io_backend='disk',
        key='gt',
        flag='unchanged'),
    dict(type='RescaleToZeroOne', keys=['lq', 'gt']),
    dict(
        type='Normalize',
        keys=['lq', 'gt'],
        mean=[0, 0, 0],
        std=[1, 1, 1],
        to_rgb=True),
    dict(type='PairedRandomCrop', gt_patch_size=128),
    dict(
 

Given a configuration that trains a model on the whole DIV2K, we need to modify the configuration so that it trains on our tiny dataset.

In [23]:
from mmcv.runner import set_random_seed

# Training folders
cfg.data.train.dataset.lq_folder = './data/DIV2K/Train/LR_x4'
cfg.data.train.dataset.gt_folder = './data/DIV2K/Train/HR'
cfg.data.train.dataset.ann_file = './data/DIV2K/Train/training_ann.txt'

# Validation folders
cfg.data.val.lq_folder = './data/DIV2K/Val/LR_x4'
cfg.data.val.gt_folder = './data/DIV2K/Val/HR'

# Test folders
cfg.data.test.lq_folder = './data/DIV2K/Val/LR_x4'
cfg.data.test.gt_folder = './data/DIV2K/Val/HR'
# cfg.data.test.lq_folder = './data/DIV2K/Test/LR_x4'
# cfg.data.test.gt_folder = './data/DIV2K/Test/HR'

# Use the pre-trained SRCNN model
#cfg.load_from = './checkpoint/srcnn_x4k915_1x16_1000k_div2k_20200608-4186f232.pth'

exp_name = 'srresnet_base'

# Set up working dir to save files and logs
cfg.work_dir = f'./p2_exps/{exp_name}'

# Use smaller batch size for training
cfg.data.samples_per_gpu = 8 #4
cfg.data.workers_per_gpu = 0
cfg.data.val_workers_per_gpu = 0

# Reduce the number of iterations
cfg.total_iters = 1000000 #200

# Training scheme
cfg.lr_config = {}
cfg.lr_config.policy = 'CosineRestart' #'Step'
cfg.lr_config.by_epoch = False
cfg.lr_config.periods = [250000, 250000, 250000, 250000]
# cfg.lr_config.step = [100]
cfg.lr_config.restart_weights=[1, 1, 1, 1]
cfg.lr_config.min_lr=1e-7
#cfg.lr_config.gamma = 0.5 'Step'

# Evaluate every 5000 iterations
cfg.evaluation.interval = 1000 #20 5000
cfg.evaluation.save_image = True 
cfg.evaluation.gpu_collect = True 
# if cfg.evaluation.get('gpu_collect', None):
#   cfg.evaluation.pop('gpu_collect')

# Save the checkpoints every N iterations
cfg.checkpoint_config.interval = 5000 #20
cfg.checkpoint_config.save_optimizer = True
cfg.checkpoint_config.by_epoch = False

# Print out the log every N iterations
cfg.log_config.interval = 100 #5

# Set seed thus the results are reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpus = 1

### **Train**

FInally, let's initialize the dataset and restorer, and train a new restorer!

In [25]:
import os.path as osp

from mmedit.datasets import build_dataset
from mmedit.models import build_model
from mmedit.apis import train_model
from mmcv.runner import init_dist

import mmcv
import os

# Initialize distributed training (only need to initialize once), comment it if
# have already run this part
os.environ['RANK'] = '0'
os.environ['WORLD_SIZE'] = '1'
os.environ['MASTER_ADDR'] = '127.0.0.1'
os.environ['MASTER_PORT'] = '50297'
try:
    init_dist('pytorch', **cfg.dist_params)
except:
    print("ERROR init_dist")


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the SRCNN model
model = build_model(
        cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

# Meta information
meta = dict()
if cfg.get('exp_name', None) is None:
    cfg['exp_name'] = osp.splitext(osp.basename(cfg.work_dir))[0]
meta['exp_name'] = cfg.exp_name
meta['mmedit Version'] = mmedit.__version__
meta['seed'] = 0

model

ERROR init_dist


BasicRestorer(
  (generator): MSRResNet(
    (conv_first): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (trunk_net): Sequential(
      (0): ResidualBlockNoBN(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
      )
      (1): ResidualBlockNoBN(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
      )
      (2): ResidualBlockNoBN(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
      )
      (3): ResidualBlockNoBN(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): 

In [26]:
print(f"num of parameters in model: \t {sum(p.numel() for p in model.parameters())}")
print(f"num of parameters we can use: \t 1821085")

num of parameters in model: 	 1517571
num of parameters we can use: 	 1821085


In [ ]:
# Train the model
train_model(model, datasets, cfg, distributed=True, validate=True, meta=meta)

2020-11-05 13:52:05,830 - mmedit - INFO - Start running, host: root@f912b67752c0, work_dir: /content/mmediting/p2_exps/srresnet_base
INFO:mmedit:Start running, host: root@f912b67752c0, work_dir: /content/mmediting/p2_exps/srresnet_base
2020-11-05 13:52:05,832 - mmedit - INFO - workflow: [('train', 1)], max: 1000000 iters
INFO:mmedit:workflow: [('train', 1)], max: 1000000 iters
2020-11-05 13:53:50,622 - mmedit - INFO - Iter [100/1000000]	lr_generator: 2.000e-04, eta: 12 days, 2:59:15, time: 1.048, data_time: 0.994, memory: 701, loss_pix: 0.0321, loss: 0.0321
INFO:mmedit:Iter [100/1000000]	lr_generator: 2.000e-04, eta: 12 days, 2:59:15, time: 1.048, data_time: 0.994, memory: 701, loss_pix: 0.0321, loss: 0.0321
2020-11-05 13:55:34,875 - mmedit - INFO - Iter [200/1000000]	lr_generator: 2.000e-04, eta: 12 days, 2:14:41, time: 1.043, data_time: 0.989, memory: 701, loss_pix: 0.0299, loss: 0.0299
INFO:mmedit:Iter [200/1000000]	lr_generator: 2.000e-04, eta: 12 days, 2:14:41, time: 1.043, data_t

### **Understand the log**

From the log, we can have a basic understanding on the training process and know how well the restorer is trained.

Firstly, the SRCNN pre-trained on DIV2K using bicubic downsampling kernel is loaded. During training you can keep track on the training status, including the current iteration, learning rate, losses, and time per iteration. 

$\quad$

Also, evaluation will be done from time to time. From the log we can see that the PSNR keeps increasing, up to 25.9744 dB in just 100 iterations. Here PSNR and SSIM are metrics measuring the "fidelity" of the output to the groundtruth. Higher values indicate better results.

Not bad!

### **Test the finetuned SRCNN**

In [ ]:
from mmedit.apis import single_gpu_test
from mmedit.datasets import build_dataloader
from mmcv.parallel import MMDataParallel


# Build a test dataloader and model
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])

# Perform the test with single gpu. Saving result images by setting
# save_image and save_path arguments. The two arguments will be passed
# to model.forword_test() where images are saved.
# See https://github.com/open-mmlab/mmediting/blob/8b5c0c5f49e60fd6ab0503031b62dee7832faf72/mmedit/models/mattors/indexnet.py#L72.
outputs = single_gpu_test(model, data_loader, save_image=True,
                          save_path=f'./p2_exps/{exp_name}/results') #'./p2_exps/srresnet_srgan/results'

# Pop out some unnecessary arguments
eval_config = cfg.evaluation
eval_config.pop('interval')
eval_config.pop('save_image', False)
eval_config.pop('save_path', None)

eval_res = dataset.evaluate(outputs, **eval_config)
print()  # endline of progress bar
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

The result is the same as the last validation! This is because we use the same data for test and validation. Next, let's take a look at the visual results of the test! The results are saved in the directory tutorial_exps/srcnn/results that we specified.

In [ ]:
!ls ./p2_exps/srresnet_srgan/results

In [ ]:
import cv2
import matplotlib.pyplot as plt
import mmcv
import torch
import torchvision

from mmedit.models import build_model
from mmcv.runner import load_checkpoint

# Read the images
img_SR = mmcv.imread('/content/mmediting/data/DIV2K/Val/LR_x4/0002.png', channel_order='rgb')
img_SR_ft = mmcv.imread(f'./p2_exps/{exp_name}/results/0002.png', channel_order='rgb')
img_HR = mmcv.imread('/content/mmediting/data/DIV2K/Val/HR/0002.png', channel_order='rgb')


# Let's compare the result with the LR and HR images
fig = plt.figure(figsize=(18, 16))
ax1 = fig.add_subplot(1, 3, 1)  # 2 rows, 2 columns, the top left image  
plt.title('Before finetune', fontsize=16)
ax1.axis('off')
ax2 = fig.add_subplot(1, 3, 2)  # 2 rows, 2 columns, the top right image 
plt.title('After finetune', fontsize=16)
ax2.axis('off')
ax3 = fig.add_subplot(1, 3, 3)  # 2 rows, 2 columns, the bottom right image  
plt.title('HR image', fontsize=16)
ax3.axis('off')
ax1.imshow(img_SR)
ax2.imshow(img_SR_ft)
ax3.imshow(img_HR)

## What to Do Next?

Single image super-resolution is just a part of the super-resolution family. There are more waiting for you to explore. For example, you can take a look at the following works:

1. Video super-resolution - [EDVR](https://arxiv.org/abs/1905.02716)
2. Reference-based image super-resolution - [SRNTT](https://arxiv.org/abs/1903.00834)

Enjoy!